In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time
import requests
import os
import csv
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
from splinter import Browser
from selenium.webdriver.chrome.service import Service

my_service = Service(executable_path=r'C:\Users\pc\OneDrive\Desktop\TCG-scraper\chromedriver.exe')
browser = Browser('chrome', service=my_service)

In [6]:
# visiting the webpage

url = 'https://shop.tcgplayer.com/sellers'
browser.visit(url)

In [7]:
# xpath for the next button at the end of each page
# //*[@id="app"]/div/div/section[2]/section/section/section/div[2]/div[1]/a[2]/span/span/svg

# text_xpath = '//*[@id="SellerName"]'
seller = 'GAMELAND'
seller_list = [seller, 'xyz', 'abc']


# passing in the seller name

browser.fill('SellerName', seller)

In [8]:
browser.find_by_xpath('//*[@id="maincontentinnerpadding"]/div/div/form/div/ul/li[4]/div/input').click()
#time.sleep(3)
browser.find_by_xpath('//*[@id="maincontentinnerpadding"]/div/div[2]/div[3]/div/div[3]/button').click()
time.sleep(3)

In [9]:
price_list = []

In [10]:
#search-result__title
#inventory__price-with-shipping
#search-result__market-price--value


for i in range(30):
    req = browser.html
    soup = BeautifulSoup(req, "html.parser")

    cat_tags = soup.find_all('h3', class_ ="search-result__category-name")
    set_tags = soup.find_all('h4', class_ ="search-result__subtitle")
    title_tags = soup.find_all('span', class_ = "search-result__title")
    price1_tags = soup.find_all('span', class_ = "inventory__price-with-shipping")
    price2_tags = soup.find_all('span', class_ = "search-result__market-price--value")

    for  cat, sset, titles, price1, price2 in zip( cat_tags ,set_tags, title_tags, price1_tags, price2_tags):
        row = []
        row.append(cat.text)
        row.append(sset.text)
        row.append(titles.text)
        row.append(price1.text)
        row.append(price2.text)
        price_list.append(row)           
    
    browser.find_by_xpath('/html/body/div[2]/div/div/section[2]/section/section/section/div[2]/div[1]/a[2]').click()
    time.sleep(4)
    
    
                
               

In [11]:
browser.quit

<bound method BaseWebDriver.quit of <splinter.driver.webdriver.chrome.WebDriver object at 0x000001F551439250>>

In [14]:
df = pd.DataFrame(price_list)
df

,0,1,2,3,4
0,Magic: The Gathering,The Lost Caverns of Ixalan,Swamp (0289),$1.12,$0.41
1,Magic: The Gathering,Theros Beyond Death,Island,$1.00,$1.62
2,Magic: The Gathering,Wilds of Eldraine,Up the Beanstalk,$3.50,$4.23
3,Magic: The Gathering,The Lost Caverns of Ixalan,"Kutzil, Malamet Exemplar (Showcase)",$1.40,$0.30
4,Magic: The Gathering,Theros Beyond Death,Swamp,$1.75,$1.54
...,...,...,...,...,...
715,Magic: The Gathering,Jumpstart 2022,Swamp (106),$0.25,$0.06
716,Magic: The Gathering,The Brothers' War,Mishra's Command,$0.25,$0.11
717,Magic: The Gathering,The Lost Caverns of Ixalan,Mischievous Pup,$0.20,$0.08
718,Magic: The Gathering,The Lost Caverns of Ixalan,Magmatic Galleon (Extended Art),$2.55,$0.57


In [15]:
df.columns = [  "category", "set", 'Name', 'Listed Price','Market Price']


In [11]:
# reformat data


df["Market Price"] = df["Market Price"].str.replace("$", "")
df["Listed Price"] = df["Listed Price"].str.replace("$", "")
df['Market Price'] = df['Market Price'].astype(float)
df['Listed Price'] = df['Listed Price'].astype(float)



C:\Users\pc\AppData\Local\Temp\ipykernel_4036\4131323524.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["Market Price"] = df["Market Price"].str.replace("$", "")
C:\Users\pc\AppData\Local\Temp\ipykernel_4036\4131323524.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["Listed Price"] = df["Listed Price"].str.replace("$", "")


In [12]:
df["over_under"] = (df["Listed Price"]) - (df['Market Price'])


In [13]:
sorted_df = df.sort_values(["over_under"])
sorted_df.head(5)

,category,set,Name,Listed Price,Market Price,over_under
210,Magic: The Gathering,Shadows over Innistrad,Cryptolith Rite,6.70,13.45,-6.75
366,Magic: The Gathering,Strixhaven: School of Mages,Crackle with Power,4.16,10.35,-6.19
221,Magic: The Gathering,Strixhaven: School of Mages,Wandering Archaic,1.90,5.83,-3.93
592,Magic: The Gathering,Magic 2012 (M12),Jace's Archivist,1.49,5.38,-3.89
242,Magic: The Gathering,Journey Into Nyx,Dictate of Erebos,6.90,10.54,-3.64


In [14]:
limit_df= sorted_df.loc[sorted_df['category'] == "Magic: The Gathering"]
limit_df= limit_df.iloc[:100]
limit_df

,category,set,Name,Listed Price,Market Price,over_under
210,Magic: The Gathering,Shadows over Innistrad,Cryptolith Rite,6.70,13.45,-6.75
366,Magic: The Gathering,Strixhaven: School of Mages,Crackle with Power,4.16,10.35,-6.19
221,Magic: The Gathering,Strixhaven: School of Mages,Wandering Archaic,1.90,5.83,-3.93
592,Magic: The Gathering,Magic 2012 (M12),Jace's Archivist,1.49,5.38,-3.89
242,Magic: The Gathering,Journey Into Nyx,Dictate of Erebos,6.90,10.54,-3.64
...,...,...,...,...,...,...
463,Magic: The Gathering,Dominaria United,The Raven Man,0.20,0.30,-0.10
166,Magic: The Gathering,Kamigawa: Neon Dynasty,Spirited Companion,0.25,0.35,-0.10
597,Magic: The Gathering,Commander Masters,Darksteel Monolith,7.49,7.59,-0.10
287,Magic: The Gathering,Phyrexia: All Will Be One,Soulless Jailer,1.00,1.09,-0.09


In [15]:
limit_df.to_csv(f"{seller}_report.csv")